<a href="https://colab.research.google.com/github/tranvangaohd1994/STP_2021_2/blob/LinhDK/feature_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications.vgg16 import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/DeepLearning/STP-2019/flower-dataset.zip"

Archive:  /content/drive/MyDrive/Colab Notebooks/DeepLearning/STP-2019/flower-dataset.zip
   creating: dataset/
   creating: dataset/Bluebell/
  inflating: dataset/Bluebell/image_0241.jpg  
  inflating: dataset/Bluebell/image_0242.jpg  
  inflating: dataset/Bluebell/image_0243.jpg  
  inflating: dataset/Bluebell/image_0244.jpg  
  inflating: dataset/Bluebell/image_0245.jpg  
  inflating: dataset/Bluebell/image_0246.jpg  
  inflating: dataset/Bluebell/image_0247.jpg  
  inflating: dataset/Bluebell/image_0248.jpg  
  inflating: dataset/Bluebell/image_0249.jpg  
  inflating: dataset/Bluebell/image_0250.jpg  
  inflating: dataset/Bluebell/image_0251.jpg  
  inflating: dataset/Bluebell/image_0252.jpg  
  inflating: dataset/Bluebell/image_0253.jpg  
  inflating: dataset/Bluebell/image_0254.jpg  
  inflating: dataset/Bluebell/image_0255.jpg  
  inflating: dataset/Bluebell/image_0256.jpg  
  inflating: dataset/Bluebell/image_0257.jpg  
  inflating: dataset/Bluebell/image_0258.jpg  
  inflating

In [ ]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('dataset/'))
print(image_path)
# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)
print(image_path)

['dataset/Tigerlily/image_0553.jpg', 'dataset/Tigerlily/image_0484.jpg', 'dataset/Tigerlily/image_0507.jpg', 'dataset/Tigerlily/image_0520.jpg', 'dataset/Tigerlily/image_0557.jpg', 'dataset/Tigerlily/image_0505.jpg', 'dataset/Tigerlily/image_0552.jpg', 'dataset/Tigerlily/image_0517.jpg', 'dataset/Tigerlily/image_0515.jpg', 'dataset/Tigerlily/image_0558.jpg', 'dataset/Tigerlily/image_0495.jpg', 'dataset/Tigerlily/image_0549.jpg', 'dataset/Tigerlily/image_0498.jpg', 'dataset/Tigerlily/image_0486.jpg', 'dataset/Tigerlily/image_0538.jpg', 'dataset/Tigerlily/image_0546.jpg', 'dataset/Tigerlily/image_0532.jpg', 'dataset/Tigerlily/image_0483.jpg', 'dataset/Tigerlily/image_0537.jpg', 'dataset/Tigerlily/image_0551.jpg', 'dataset/Tigerlily/image_0526.jpg', 'dataset/Tigerlily/image_0516.jpg', 'dataset/Tigerlily/image_0497.jpg', 'dataset/Tigerlily/image_0548.jpg', 'dataset/Tigerlily/image_0503.jpg', 'dataset/Tigerlily/image_0490.jpg', 'dataset/Tigerlily/image_0556.jpg', 'dataset/Tigerlily/image_05

In [ ]:
# Đường dẫn ảnh sẽ là ...dataset/tên_loài_hoa/tên_ảnh ví dụ ...dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels = [p.split(os.path.sep)[-2] for p in image_path]
print(labels)
# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)
print(labels)

['Snowdrop', 'Sunflower', 'Tulip', 'Buttercup', 'Windflower', 'Daisy', 'Crocus', 'Bluebell', 'Windflower', 'Buttercup', 'Snowdrop', 'ColtsFoot', 'Buttercup', 'Daffodil', 'Bluebell', 'Sunflower', 'Daisy', 'Daffodil', 'LilyValley', 'LilyValley', 'Buttercup', 'Dandelion', 'Pansy', 'ColtsFoot', 'Cowslip', 'Snowdrop', 'Pansy', 'Tigerlily', 'Snowdrop', 'Windflower', 'Cowslip', 'Windflower', 'ColtsFoot', 'Snowdrop', 'LilyValley', 'ColtsFoot', 'Tigerlily', 'Daisy', 'LilyValley', 'Cowslip', 'Sunflower', 'Daisy', 'Dandelion', 'Tigerlily', 'Iris', 'Fritillary', 'Tigerlily', 'Daisy', 'Iris', 'Windflower', 'Buttercup', 'Dandelion', 'Daisy', 'Pansy', 'Buttercup', 'ColtsFoot', 'ColtsFoot', 'Pansy', 'LilyValley', 'Pansy', 'Fritillary', 'Fritillary', 'Fritillary', 'Snowdrop', 'Buttercup', 'ColtsFoot', 'Tulip', 'Fritillary', 'Crocus', 'Snowdrop', 'Bluebell', 'Windflower', 'Iris', 'Pansy', 'Snowdrop', 'Windflower', 'Fritillary', 'Tulip', 'Bluebell', 'Tulip', 'Crocus', 'Cowslip', 'Cowslip', 'Snowdrop', 'C

In [ ]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
model = VGG16(weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
list_image = np.vstack(list_image)

In [ ]:
# Dùng pre-trained model để lấy ra các feature của ảnh
features = model.predict(list_image)

# Giống bước flatten trong CNN, chuyển từ tensor 3 chiều sau ConvNet sang vector 1 chiều
features = features.reshape((features.shape[0], 512*7*7))

In [ ]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# Grid search để tìm các parameter tốt nhất cho model. C = 1/lamda, hệ số trong regularisation. Solver là kiểu optimize
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
params = {'C' : [0.1, 1.0, 10.0, 100.0]}
#model = GridSearchCV(LogisticRegression(solver='lbfgs', multi_class='multinomial'), params)
model = GridSearchCV(LogisticRegression(), params)
model.fit(X_train, y_train)
print('Best parameter for the model {}'.format(model.best_params_))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Best parameter for the model {'C': 100.0}


In [ ]:
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       0.92      0.92      0.92        13
           2       1.00      0.90      0.95        20
           3       0.68      0.93      0.79        14
           4       1.00      1.00      1.00        13
           5       1.00      0.74      0.85        19
           6       0.87      1.00      0.93        13
           7       1.00      0.88      0.93        16
           8       1.00      0.91      0.95        22
           9       0.95      1.00      0.97        18
          10       1.00      1.00      1.00        18
          11       1.00      0.94      0.97        18
          12       0.71      1.00      0.83        12
          13       1.00      1.00      1.00        13
          14       1.00      1.00      1.00        19
          15       0.85      0.85      0.85        13
          16       1.00      0.88      0.93        16

    accuracy              